In [ ]:
# -*- coding: utf-8 -*-
# Jupyter cell — LOCO + chr-disjoint HPO with expanded spaces for CatBoost/XGBoost

import os, json, random, warnings
warnings.filterwarnings("ignore")

import optuna
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from functools import partial
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor

# ========== config ==========
SEED = 42
random.seed(SEED); np.random.seed(SEED)

TRAIN_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X1_all_rank_features_with_y.tsv"
VAL_PATH   = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X2_all_rank_features_with_y.tsv"

SAVE_ROOT = "../results/hpo_loco_chr_disjoint"
os.makedirs(SAVE_ROOT, exist_ok=True)

META_COLS = ["gene_name","chr","gene_start","gene_end","TSS_start","TSS_end","strand","gex","gex_rank"]
TARGET_R  = "gex_rank"
TARGET_B  = "gex_binary"

# trials per study（依資源調整）
N_TRIALS_LGBM_BIN = 300
N_TRIALS_XGB_BIN  = 300
N_TRIALS_CAT_BIN  = 300
N_TRIALS_CAT_REG  = 300

K_VAL_CHR = 4  # 每個 trial 驗證用的 chr 數

# ========== load ==========
df_x1 = pd.read_csv(TRAIN_PATH, sep="\t"); df_x1["cell_id"] = "X1"
df_x2 = pd.read_csv(VAL_PATH,   sep="\t"); df_x2["cell_id"] = "X2"

df_all = pd.concat([df_x1, df_x2], ignore_index=True)
df_all[TARGET_B] = (df_all["gex"] > 0.0).astype(int)

# features
drop_cols = set(META_COLS + [TARGET_B, "cell_id"])
feature_cols = [c for c in df_all.columns if c not in drop_cols]
chromosomes = sorted(df_all["chr"].unique())
print(f"🧬 chromosomes: {chromosomes}")
print(f"Feature count: {len(feature_cols)} | Total rows: {len(df_all):,}")

# ========== helpers ==========
def split_loco_by_chromosomes(trial, train_cell, val_cell, k_val=K_VAL_CHR):
    """LOCO: train on one cell, validate on the other; chromosomes disjoint."""
    rng = random.Random(SEED)  # 確保每個 trial 不同
    if k_val > len(chromosomes):
        raise ValueError("k_val exceeds available chromosomes.")
    val_chrs = rng.sample(chromosomes, k_val)
    tr = df_all[(df_all["cell_id"] == train_cell) & (~df_all["chr"].isin(val_chrs))].copy()
    va = df_all[(df_all["cell_id"] == val_cell)   & ( df_all["chr"].isin(val_chrs))].copy()
    return tr, va, val_chrs

def mean_per_chr_auc(df_v, y_true, y_pred):
    per = []
    tmp = df_v.assign(_y=np.asarray(y_true), _p=np.asarray(y_pred))
    for _, g in tmp.groupby("chr"):
        yt = g["_y"].values; yp = g["_p"].values
        if len(np.unique(yt)) == 2 and len(yt) >= 3:
            per.append(roc_auc_score(yt, yp))
    return np.nanmean(per) if per else np.nan

def mean_per_chr_spearman(df_v, y_true, y_pred):
    per = []
    tmp = df_v.assign(_y=np.asarray(y_true), _p=np.asarray(y_pred))
    for _, g in tmp.groupby("chr"):
        if len(g) >= 3:
            per.append(spearmanr(g["_y"], g["_p"])[0])
    return np.nanmean(per) if per else np.nan

# ========== objectives ==========
def objective_lgbm_bin(trial, train_cell, val_cell):
    tr, va, _ = split_loco_by_chromosomes(trial, train_cell, val_cell)
    X_tr, y_tr = tr[feature_cols], tr[TARGET_B]
    X_va, y_va = va[feature_cols], va[TARGET_B]
{"objective": "binary",        "verbosity": -1,
        "seed": SEED,
        "deterministic": True,
        "force_row_wise": True,
        "metric": "auc",'boosting_type': 'gbdt', 'learning_rate': 0.014346720117475667, 'num_leaves': 255, 'max_depth': 6, 'feature_fraction': 0.9952944676298193, 'bagging_fraction': 0.8216668173082509, 'bagging_freq': 3, 'lambda_l1': 0.018487314424426086, 'lambda_l2': 3.362256989326086, 'min_gain_to_split': 0.00014663142338152378, 'min_data_in_leaf': 80}. Best is trial 247 with value: 0.9145096322649733.

    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255, step=16),
        "max_depth": trial.suggest_int("max_depth", -1, 12),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 2.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.0, 0.1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 200),
        "verbosity": -1,
        "seed": SEED,
        "deterministic": True,
        "force_row_wise": True,
    }
    dtr = lgb.Dataset(X_tr, label=y_tr); dva = lgb.Dataset(X_va, label=y_va, reference=dtr)
    mdl = lgb.train(params, dtr, valid_sets=[dva], num_boost_round=6000,
                    callbacks=[lgb.early_stopping(200, verbose=False)])
    p = mdl.predict(X_va, num_iteration=mdl.best_iteration)
    auc_chr = mean_per_chr_auc(va, y_va, p)
    auc_all = roc_auc_score(y_va, p)
    return auc_chr if not np.isnan(auc_chr) else auc_all

# ========== run studies for both LOCO directions ==========
def run_and_save(name, objective_fn, n_trials, train_cell, val_cell):
    tag = f"{name}_{train_cell}_to_{val_cell}"
    print(f"\n===== 🔎 Start study: {tag} | trials={n_trials} =====")
    study = optuna.create_study(direction="maximize",
                                sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(partial(objective_fn, train_cell=train_cell, val_cell=val_cell),
                   n_trials=n_trials, show_progress_bar=True)
    print(f"✅ {tag} Best value: {study.best_value}")
    print("🏆 Best params:")
    for k,v in study.best_params.items():
        print(f"  {k}: {v}")
    out_dir = os.path.join(SAVE_ROOT, tag); os.makedirs(out_dir, exist_ok=True)
    study.trials_dataframe().to_csv(os.path.join(out_dir, "trials_log.csv"), index=False)
    with open(os.path.join(out_dir, "best_params.json"), "w") as f:
        json.dump(study.best_params, f, indent=4)
    print(f"💾 Saved to {out_dir}")
    return study

PAIRS = [("X1","X2"), ("X2","X1")]

for a,b in PAIRS:
    run_and_save("lgbm_binary",  objective_lgbm_bin, N_TRIALS_LGBM_BIN, a, b)

[I 2025-10-23 01:53:15,918] A new study created in memory with name: no-name-d6480c2c-996c-40a7-a725-e12fc5ce67e2


🧬 chromosomes: ['chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9']
Feature count: 2892 | Total rows: 32,568

===== 🔎 Start study: lgbm_binary_X1_to_X2 | trials=300 =====


Best trial: 0. Best value: 0.905329:   0%|          | 1/300 [04:24<21:59:32, 264.79s/it]

[I 2025-10-23 01:57:40,726] Trial 0 finished with value: 0.9053291570312813 and parameters: {'boosting_type': 'dart', 'learning_rate': 0.05395030966670229, 'num_leaves': 159, 'max_depth': 1, 'feature_fraction': 0.662397808134481, 'bagging_fraction': 0.5290418060840998, 'bagging_freq': 7, 'lambda_l1': 1.2022300234864176, 'lambda_l2': 3.540362888980227, 'min_gain_to_split': 0.0020584494295802446, 'min_data_in_leaf': 195}. Best is trial 0 with value: 0.9053291570312813.


Best trial: 1. Best value: 0.90918:   1%|          | 2/300 [05:19<11:41:12, 141.18s/it] 

...

[I 2025-10-23 17:53:16,887] Trial 262 finished with value: 0.9133037382505031 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.0147334738929461, 'num_leaves': 255, 'max_depth': 6, 'feature_fraction': 0.9698231220837077, 'bagging_fraction': 0.8513692439794677, 'bagging_freq': 3, 'lambda_l1': 0.005675398260654437, 'lambda_l2': 3.2476645657017618, 'min_gain_to_split': 0.00011291012102230343, 'min_data_in_leaf': 62}. Best is trial 254 with value: 0.9155502562335951.


Best trial: 254. Best value: 0.91555:  88%|████████▊ | 264/300 [16:01:46<2:55:11, 291.99s/it]

[I 2025-10-23 17:55:02,169] Trial 263 finished with value: 0.91428291212997 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.0152511629938118, 'num_leaves': 255, 'max_depth': 6, 'feature_fraction': 0.961730257553892, 'bagging_fraction': 0.8432637993106202, 'bagging_freq': 3, 'lambda_l1': 0.007301820663386057, 'lambda_l2': 3.164713036112661, 'min_gain_to_split': 0.0005834006018741475, 'min_data_in_leaf': 56}. Best is trial 254 with value: 0.9155502562335951.


Best trial: 254. Best value: 0.91555:  88%|████████▊ | 265/300 [16:03:36<2:18:34, 237.55s/it]

[I 2025-10-23 17:56:52,713] Trial 264 finished with value: 0.9128606473014569 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.015300827875819371, 'num_leaves': 255, 'max_depth': 6, 'feature_fraction': 0.9607251365237753, 'bagging_fraction': 0.8544166105464417, 'bagging_freq': 3, 'lambda_l1': 0.0008506808390266619, 'lambda_l2': 3.151459518745016, 'min_gain_to_split': 0.0003445565498368287, 'min_data_in_leaf': 53}. Best is trial 254 with value: 0.9155502562335951.
